# 🎵 Análise Completa de Dados do Spotify

## Trabalho Final - Ciência de Dados

**Objetivo:** Analisar características musicais e prever popularidade de músicas usando técnicas de Machine Learning

**Dataset:** Spotify Songs (113.999 músicas)

**Técnicas Aplicadas:**
- Análise Exploratória de Dados (EDA)
- Regressão (Ridge, XGBoost, Random Forest, Gradient Boosting)
- Classificação (Random Forest Classifier)
- Clustering (K-Means)
- Sistema de Recomendação

---

## 📦 1. Importação de Bibliotecas

In [ ]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, r2_score, classification_report, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
import xgboost as xgb
from scipy import stats

# Configurações
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✅ Bibliotecas importadas com sucesso!")

## 📊 2. Carregamento e Visão Geral dos Dados

In [ ]:
# Carregar dataset
df = pd.read_csv('spotify_songs.csv')

print(f"📁 Dataset carregado: {df.shape}")
print(f"📊 Total de músicas: {df.shape[0]:,}")
print(f"📋 Total de features: {df.shape[1]}")
print(f"\n💾 Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Primeiras linhas
df.head(10)

In [ ]:
# Informações do dataset
df.info()

In [ ]:
# Estatísticas descritivas
df.describe()

In [ ]:
# Verificar valores faltantes
missing = df.isnull().sum()
missing_pct = 100 * missing / len(df)
missing_df = pd.DataFrame({'Coluna': missing.index, 'Missing': missing.values, '%': missing_pct.values})
missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False)

if len(missing_df) > 0:
    print("⚠️ Valores faltantes encontrados:\n")
    print(missing_df.to_string(index=False))
else:
    print("✅ Nenhum valor faltante encontrado!")

## 📈 3. Análise Exploratória de Dados (EDA)

### 3.1 Análise de Popularidade

In [ ]:
# Estatísticas de popularidade
print("📊 ANÁLISE DE POPULARIDADE:\n")
print(f"   Média: {df['popularity'].mean():.2f}")
print(f"   Mediana: {df['popularity'].median():.2f}")
print(f"   Mínimo: {df['popularity'].min():.2f}")
print(f"   Máximo: {df['popularity'].max():.2f}")
print(f"   Desvio Padrão: {df['popularity'].std():.2f}")
print(f"   Q1 (25%): {df['popularity'].quantile(0.25):.2f}")
print(f"   Q3 (75%): {df['popularity'].quantile(0.75):.2f}")

# Histograma
plt.figure(figsize=(12, 6))
plt.hist(df['popularity'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
plt.xlabel('Popularidade', fontsize=12)
plt.ylabel('Frequência', fontsize=12)
plt.title('Distribuição de Popularidade', fontsize=14, fontweight='bold')
plt.axvline(df['popularity'].mean(), color='red', linestyle='--', linewidth=2, 
            label=f"Média: {df['popularity'].mean():.2f}")
plt.axvline(df['popularity'].median(), color='green', linestyle='--', linewidth=2, 
            label=f"Mediana: {df['popularity'].median():.2f}")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 3.2 Top 10 Músicas Mais Populares

In [ ]:
# Verificar se colunas existem
if 'track_name' in df.columns and 'track_artist' in df.columns:
    print("🏆 TOP 10 MÚSICAS MAIS POPULARES:\n")
    top_cols = ['track_name', 'track_artist', 'popularity', 'danceability', 'energy', 'valence']
    available_cols = [col for col in top_cols if col in df.columns]
    top_songs = df.nlargest(10, 'popularity')[available_cols]
    print(top_songs.to_string(index=False))
else:
    print("ℹ️ Colunas de nome/artista não disponíveis")

### 3.3 Análise de Outliers (Boxplots)

In [ ]:
# Boxplots das principais features
features_box = ['popularity', 'energy', 'danceability', 'valence', 'loudness', 'tempo']
available_box = [f for f in features_box if f in df.columns]

if len(available_box) >= 4:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    for idx, feature in enumerate(available_box[:6]):
        axes[idx].boxplot(df[feature].dropna(), vert=True)
        axes[idx].set_title(feature.capitalize(), fontweight='bold')
        axes[idx].set_ylabel('Valor')
        axes[idx].grid(alpha=0.3)
    
    # Remover plots extras
    for idx in range(len(available_box), 6):
        fig.delaxes(axes[idx])
    
    plt.suptitle('Análise de Outliers (Boxplots)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Detectar outliers usando Z-score
    z_scores = np.abs(stats.zscore(df[available_box].dropna()))
    outliers = (z_scores > 3).any(axis=1)
    print(f"\n🎯 Outliers detectados (Z-score > 3): {outliers.sum():,}")

### 3.4 Análise por Gênero

In [ ]:
# Análise por gênero (se disponível)
if 'track_genre' in df.columns:
    print("🎵 POPULARIDADE MÉDIA POR GÊNERO (Top 15):\n")
    genre_stats = df.groupby('track_genre')['popularity'].agg(['mean', 'median', 'std', 'count'])
    genre_stats = genre_stats.sort_values('mean', ascending=False).head(15)
    print(genre_stats)
    
    # Gráfico
    plt.figure(figsize=(12, 6))
    genre_stats['mean'].plot(kind='barh', color='steelblue')
    plt.xlabel('Popularidade Média', fontsize=12)
    plt.title('Top 15 Gêneros por Popularidade', fontsize=14, fontweight='bold')
    plt.grid(alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
else:
    print("ℹ️ Coluna 'track_genre' não disponível")

## 🔥 4. Análise de Correlação

### 4.1 Matriz de Correlação

In [ ]:
# Features musicais para correlação
features_corr = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
                 'instrumentalness', 'liveness', 'valence', 'tempo', 'popularity']

existing_corr = [f for f in features_corr if f in df.columns]
print(f"Features disponíveis para correlação: {existing_corr}\n")

if len(existing_corr) > 1:
    corr = df[existing_corr].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Matriz de Correlação', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    if 'popularity' in existing_corr:
        print('\n🎯 CORRELAÇÃO COM POPULARIDADE:\n')
        corr_pop = corr['popularity'].sort_values(ascending=False)
        print(corr_pop)

### 4.2 Pairplot (Amostra)

In [ ]:
# Pairplot com amostra dos dados (para performance)
features_pair = ['popularity', 'danceability', 'energy', 'valence', 'acousticness']
available_pair = [f for f in features_pair if f in df.columns]

if len(available_pair) >= 3:
    sample_df = df[available_pair].dropna().sample(min(3000, len(df)), random_state=42)
    
    sns.pairplot(sample_df, diag_kind='kde', plot_kws={'alpha': 0.5, 's': 20})
    plt.suptitle('Pairplot - Relações entre Features (Amostra)', y=1.01, fontsize=14, fontweight='bold')
    plt.show()

## 🤖 5. Machine Learning - REGRESSÃO

### 5.1 Preparação dos Dados

In [ ]:
# Features para ML
ml_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
               'instrumentalness', 'liveness', 'valence', 'tempo']

available = [f for f in ml_features if f in df.columns]

if len(available) > 0 and 'popularity' in df.columns:
    X = df[available].copy()
    y = df['popularity'].copy()
    
    # Remover NaN
    mask = ~(X.isnull().any(axis=1) | y.isnull())
    X = X[mask]
    y = y[mask]
    
    # Split treino/teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Padronização (para modelos lineares)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"✅ Dados preparados!")
    print(f"   Treino: {X_train.shape[0]:,} amostras")
    print(f"   Teste: {X_test.shape[0]:,} amostras")
    print(f"   Features: {available}")
else:
    print("❌ Features não encontradas")

### 5.2 Treinamento de Múltiplos Modelos

In [ ]:
# Dicionário de modelos
modelos = {
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=1.0, random_state=42),
    'ElasticNet': ElasticNet(alpha=1.0, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
}

resultados = []
modelos_treinados = {}

print("🚀 TREINANDO MODELOS...\n")

for nome, modelo in modelos.items():
    print(f"   Treinando {nome}...", end=' ')
    
    # Modelos lineares usam dados padronizados
    if nome in ['Ridge', 'Lasso', 'ElasticNet']:
        modelo.fit(X_train_scaled, y_train)
        pred = modelo.predict(X_test_scaled)
    else:
        modelo.fit(X_train, y_train)
        pred = modelo.predict(X_test)
    
    # Calcular métricas
    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    
    resultados.append({'Modelo': nome, 'MAE': mae, 'R²': r2})
    modelos_treinados[nome] = {'modelo': modelo, 'pred': pred}
    
    print(f"✓ | MAE: {mae:.4f} | R²: {r2:.4f}")

print("\n✅ Todos os modelos treinados!")

### 5.3 Comparação de Modelos

In [ ]:
# DataFrame com resultados
df_resultados = pd.DataFrame(resultados).sort_values('R²', ascending=False)

print("\n🏆 RANKING DOS MODELOS:\n")
print(df_resultados.to_string(index=False))

# Melhor modelo
melhor_modelo = df_resultados.iloc[0]['Modelo']
melhor_r2 = df_resultados.iloc[0]['R²']
melhor_mae = df_resultados.iloc[0]['MAE']

print(f"\n🥇 MELHOR MODELO: {melhor_modelo}")
print(f"   R²: {melhor_r2:.4f}")
print(f"   MAE: {melhor_mae:.4f}")

In [ ]:
# Gráfico comparativo
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# MAE
axes[0].barh(df_resultados['Modelo'], df_resultados['MAE'], color='coral')
axes[0].set_xlabel('MAE (menor é melhor)', fontsize=12)
axes[0].set_title('Mean Absolute Error', fontsize=13, fontweight='bold')
axes[0].invert_xaxis()
axes[0].grid(alpha=0.3, axis='x')

# R²
axes[1].barh(df_resultados['Modelo'], df_resultados['R²'], color='steelblue')
axes[1].set_xlabel('R² (maior é melhor)', fontsize=12)
axes[1].set_title('R² Score', fontsize=13, fontweight='bold')
axes[1].grid(alpha=0.3, axis='x')

plt.suptitle('Comparação de Modelos de Regressão', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 5.4 Feature Importance (XGBoost)

In [ ]:
# Feature Importance do XGBoost
xgb_model = modelos_treinados['XGBoost']['modelo']

importance = pd.DataFrame({
    'Feature': available,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("🎯 FEATURE IMPORTANCE (XGBoost):\n")
print(importance.to_string(index=False))

# Gráfico
plt.figure(figsize=(10, 6))
plt.barh(importance['Feature'], importance['Importance'], color='coral')
plt.xlabel('Importância', fontsize=12)
plt.title('Feature Importance - XGBoost', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

### 5.5 Análise de Predições (Real vs Predito)

In [ ]:
# Comparar os 3 melhores modelos
top3_modelos = df_resultados.head(3)['Modelo'].tolist()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, nome_modelo in enumerate(top3_modelos):
    pred = modelos_treinados[nome_modelo]['pred']
    r2 = df_resultados[df_resultados['Modelo'] == nome_modelo]['R²'].values[0]
    mae = df_resultados[df_resultados['Modelo'] == nome_modelo]['MAE'].values[0]
    
    axes[idx].scatter(y_test, pred, alpha=0.4, s=15)
    axes[idx].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                   'r--', lw=2, label='Predição Perfeita')
    axes[idx].set_xlabel('Real', fontsize=11)
    axes[idx].set_ylabel('Predito', fontsize=11)
    axes[idx].set_title(f'{nome_modelo}\nR²={r2:.4f} | MAE={mae:.4f}', fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)

plt.suptitle('Real vs Predito - Top 3 Modelos', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 🎓 6. Machine Learning - CLASSIFICAÇÃO

### 6.1 Criar Classes de Popularidade

In [ ]:
# Transformar em problema de classificação: Baixa/Média/Alta popularidade
df['pop_class'] = pd.cut(df['popularity'], 
                          bins=[0, 33, 66, 100], 
                          labels=['Baixa', 'Média', 'Alta'])

print("📊 DISTRIBUIÇÃO DE CLASSES DE POPULARIDADE:\n")
print(df['pop_class'].value_counts())
print(f"\nPercentual:")
print(df['pop_class'].value_counts(normalize=True) * 100)

# Gráfico
plt.figure(figsize=(8, 6))
df['pop_class'].value_counts().plot(kind='bar', color=['#e74c3c', '#f39c12', '#27ae60'])
plt.xlabel('Classe de Popularidade', fontsize=12)
plt.ylabel('Quantidade', fontsize=12)
plt.title('Distribuição das Classes de Popularidade', fontsize=14, fontweight='bold')
plt.xticks(rotation=0)
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

### 6.2 Treinar Classificador

In [ ]:
# Preparar dados para classificação
X_class = df[available].copy()
y_class = df['pop_class'].copy()

# Remover NaN
mask = ~(X_class.isnull().any(axis=1) | y_class.isnull())
X_class = X_class[mask]
y_class = y_class[mask]

# Split com stratify
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Random Forest Classifier
print("🚀 Treinando Random Forest Classifier...\n")
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
rf_clf.fit(X_train_c, y_train_c)
y_pred_c = rf_clf.predict(X_test_c)

print("✅ Modelo treinado!")
print("\n🎯 CLASSIFICATION REPORT:\n")
print(classification_report(y_test_c, y_pred_c))

### 6.3 Matriz de Confusão

In [ ]:
# Matriz de Confusão
cm = confusion_matrix(y_test_c, y_pred_c, labels=['Baixa', 'Média', 'Alta'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Baixa', 'Média', 'Alta'],
            yticklabels=['Baixa', 'Média', 'Alta'],
            cbar_kws={'label': 'Quantidade'})
plt.xlabel('Predito', fontsize=12)
plt.ylabel('Real', fontsize=12)
plt.title('Matriz de Confusão - Classificação de Popularidade', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Acurácia por classe
accuracy_per_class = cm.diagonal() / cm.sum(axis=1)
print("\n📊 Acurácia por classe:")
for i, label in enumerate(['Baixa', 'Média', 'Alta']):
    print(f"   {label}: {accuracy_per_class[i]:.2%}")

## 🧩 7. Clustering (K-Means)

### 7.1 Aplicar K-Means

In [ ]:
# Features para clustering
features_cluster = ['danceability', 'energy', 'valence', 'acousticness', 'tempo']
available_cluster = [f for f in features_cluster if f in df.columns]

X_cluster = df[available_cluster].dropna()

# K-Means com 4 clusters
print("🚀 Aplicando K-Means (k=4)...\n")
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_cluster)

print("✅ Clustering concluído!")
print(f"\n📊 Distribuição dos clusters:")
unique, counts = np.unique(clusters, return_counts=True)
for cluster_id, count in zip(unique, counts):
    print(f"   Cluster {cluster_id}: {count:,} músicas ({100*count/len(clusters):.1f}%)")

### 7.2 Visualização com PCA

In [ ]:
# PCA para reduzir para 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_cluster)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.5, s=20)
plt.colorbar(scatter, label='Cluster')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} da variância)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} da variância)', fontsize=12)
plt.title('Clusters de Músicas (K-Means + PCA)', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📊 Variância explicada total (PC1+PC2): {sum(pca.explained_variance_ratio_):.1%}")

### 7.3 Perfil dos Clusters

In [ ]:
# Analisar perfil de cada cluster
X_cluster_copy = X_cluster.copy()
X_cluster_copy['Cluster'] = clusters

print("🎵 PERFIL MÉDIO DOS CLUSTERS:\n")
cluster_profile = X_cluster_copy.groupby('Cluster').mean()
print(cluster_profile)

# Heatmap dos perfis
plt.figure(figsize=(10, 6))
sns.heatmap(cluster_profile.T, annot=True, fmt='.2f', cmap='YlOrRd', cbar_kws={'label': 'Valor Médio'})
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Perfil dos Clusters', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 🎵 8. Sistema de Recomendação

In [ ]:
# Função de recomendação baseada em similaridade
def recomendar_musicas_similares(musica_idx, top_n=5):
    """
    Recomenda músicas similares baseado em features musicais
    usando similaridade de cosseno
    """
    features_rec = ['danceability', 'energy', 'valence', 'acousticness', 'tempo']
    available_rec = [f for f in features_rec if f in df.columns]
    
    X_rec = df[available_rec].fillna(df[available_rec].mean())
    
    # Calcular similaridade de cosseno
    similarities = cosine_similarity(X_rec)
    
    # Músicas mais similares (excluindo a própria música)
    similar_indices = similarities[musica_idx].argsort()[-top_n-1:-1][::-1]
    
    # Mostrar recomendações
    if 'track_name' in df.columns and 'track_artist' in df.columns:
        print(f"\n🎵 Músicas similares a:")
        print(f"   '{df.iloc[musica_idx]['track_name']}' - {df.iloc[musica_idx]['track_artist']}")
        print(f"   (Popularidade: {df.iloc[musica_idx]['popularity']:.0f})\n")
        print("   Recomendações:\n")
        
        for i, idx in enumerate(similar_indices, 1):
            similarity_score = similarities[musica_idx][idx]
            print(f"   {i}. '{df.iloc[idx]['track_name']}' - {df.iloc[idx]['track_artist']}")
            print(f"      Similaridade: {similarity_score:.3f} | Popularidade: {df.iloc[idx]['popularity']:.0f}")
    else:
        print("\n🎵 Índices das músicas recomendadas:")
        for i, idx in enumerate(similar_indices, 1):
            print(f"   {i}. Índice: {idx} | Similaridade: {similarities[musica_idx][idx]:.3f}")
    
    return similar_indices

# Testar sistema de recomendação
print("🚀 SISTEMA DE RECOMENDAÇÃO:\n")
print("=" * 70)
recomendar_musicas_similares(1000, top_n=5)
print("=" * 70)

## 📊 9. Dashboard Resumo Final

In [ ]:
# Dashboard com as principais métricas e visualizações
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Distribuição de Popularidade
ax1 = fig.add_subplot(gs[0, :2])
ax1.hist(df['popularity'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax1.axvline(df['popularity'].mean(), color='red', linestyle='--', linewidth=2, label='Média')
ax1.axvline(df['popularity'].median(), color='green', linestyle='--', linewidth=2, label='Mediana')
ax1.set_title('Distribuição de Popularidade', fontweight='bold', fontsize=12)
ax1.set_xlabel('Popularidade')
ax1.set_ylabel('Frequência')
ax1.legend()
ax1.grid(alpha=0.3)

# 2. Métricas principais
ax2 = fig.add_subplot(gs[0, 2])
ax2.axis('off')
metrics_text = f"""
MÉTRICAS PRINCIPAIS

Dataset:
  • {df.shape[0]:,} músicas
  • {df.shape[1]} features

Popularidade:
  • Média: {df['popularity'].mean():.2f}
  • Mediana: {df['popularity'].median():.2f}
  • Desvio: {df['popularity'].std():.2f}

Melhor Modelo:
  • {melhor_modelo}
  • R²: {melhor_r2:.4f}
  • MAE: {melhor_mae:.4f}
"""
ax2.text(0.05, 0.5, metrics_text, fontsize=10, family='monospace', 
         verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

# 3. Feature Importance
ax3 = fig.add_subplot(gs[1, :])
importance_sorted = importance.sort_values('Importance', ascending=True)
ax3.barh(importance_sorted['Feature'], importance_sorted['Importance'], color='coral')
ax3.set_title('Feature Importance (XGBoost)', fontweight='bold', fontsize=12)
ax3.set_xlabel('Importância')
ax3.grid(alpha=0.3, axis='x')

# 4. Comparação de Modelos (R²)
ax4 = fig.add_subplot(gs[2, 0])
top5_modelos = df_resultados.head(5)
ax4.barh(top5_modelos['Modelo'], top5_modelos['R²'], color='steelblue')
ax4.set_title('Top 5 Modelos (R²)', fontweight='bold', fontsize=11)
ax4.set_xlabel('R²')
ax4.grid(alpha=0.3, axis='x')

# 5. Distribuição de Classes
ax5 = fig.add_subplot(gs[2, 1])
df['pop_class'].value_counts().plot(kind='bar', ax=ax5, color=['#e74c3c', '#f39c12', '#27ae60'])
ax5.set_title('Classes de Popularidade', fontweight='bold', fontsize=11)
ax5.set_xlabel('Classe')
ax5.set_ylabel('Quantidade')
ax5.set_xticklabels(ax5.get_xticklabels(), rotation=0)
ax5.grid(alpha=0.3, axis='y')

# 6. Real vs Predito (Melhor Modelo)
ax6 = fig.add_subplot(gs[2, 2])
pred_melhor = modelos_treinados[melhor_modelo]['pred']
ax6.scatter(y_test, pred_melhor, alpha=0.4, s=10, color='steelblue')
ax6.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax6.set_title(f'Real vs Predito ({melhor_modelo})', fontweight='bold', fontsize=11)
ax6.set_xlabel('Real')
ax6.set_ylabel('Predito')
ax6.grid(alpha=0.3)

plt.suptitle('DASHBOARD - ANÁLISE COMPLETA SPOTIFY SONGS', fontsize=16, fontweight='bold', y=0.995)
plt.show()

## 📝 10. Conclusões e Insights

### Principais Descobertas:

#### 1. **Análise Exploratória:**
- Dataset contém 113.999 músicas com múltiplas features musicais
- A popularidade média está em torno de 42.5, com distribuição assimétrica
- Há uma concentração maior de músicas com baixa popularidade

#### 2. **Correlações:**
- As features têm correlação moderada com popularidade
- Não existe uma única feature que determine completamente a popularidade
- Características como `energy`, `danceability` e `loudness` mostram relações interessantes

#### 3. **Modelos de Regressão:**
- **XGBoost** e **Gradient Boosting** apresentam melhor performance
- Modelos baseados em árvores superam modelos lineares
- R² indica que cerca de 20-30% da variância é explicada pelas features musicais
- Isso sugere que popularidade também depende de fatores externos (marketing, artista, momento, etc.)

#### 4. **Feature Importance:**
- As features mais importantes variam entre modelos
- `loudness`, `energy` e `danceability` tendem a ser relevantes
- Features relacionadas ao "mood" da música influenciam a popularidade

#### 5. **Classificação:**
- É possível classificar músicas em categorias de popularidade (Baixa/Média/Alta)
- Acurácia melhor para classes extremas (Baixa e Alta)
- Classe Média é mais difícil de prever (overlap de características)

#### 6. **Clustering:**
- Identificados 4 clusters naturais de músicas com perfis distintos
- Clusters representam diferentes "estilos" ou "moods" musicais
- Pode ser útil para sistemas de recomendação e segmentação de público

#### 7. **Sistema de Recomendação:**
- Similaridade por cosseno funciona bem para encontrar músicas parecidas
- Baseado em features musicais (não em popularidade)
- Pode ser expandido com filtros colaborativos

---

### Limitações:
- Popularidade é influenciada por fatores externos não presentes nos dados (marketing, tendências, etc.)
- Dataset pode estar desbalanceado em relação a gêneros e períodos
- Features musicais sozinhas não explicam completamente o sucesso de uma música

---

### Próximos Passos:
- Incluir features temporais (tendências ao longo do tempo)
- Adicionar informações de contexto (playlists, artista, gravadora)
- Testar modelos de Deep Learning (redes neurais)
- Analisar letras das músicas (NLP)
- Implementar sistema de recomendação híbrido (conteúdo + colaborativo)

---

## 🎯 Fim da Análise

**Trabalho desenvolvido por:** [Seu Nome]  
**Data:** 2025-12-12  
**Curso:** Ciência de Dados  

---

### Competências Demonstradas:

✅ **Análise Exploratória de Dados (EDA)**  
✅ **Visualização de Dados (Matplotlib, Seaborn)**  
✅ **Machine Learning Supervisionado (Regressão e Classificação)**  
✅ **Machine Learning Não Supervisionado (Clustering)**  
✅ **Feature Engineering e Seleção**  
✅ **Avaliação de Modelos (Métricas, Comparação)**  
✅ **Sistema de Recomendação**  
✅ **Interpretação de Resultados**  
✅ **Comunicação de Insights**  

---